# Homework 9: Getting Familiar with NASA Polynomials
## Due Date:  Tuesday, November 7th at 11:59 PM

Read the NASA Polynomial dataset in raw format and parse and store the data into an .xml file.

### Review of the NASA Polynomials
You can find the NASA Polynomial file in `thermo.txt`.

You can find some details on the NASA Polynomials [at this site](http://combustion.berkeley.edu/gri_mech/data/nasa_plnm.html) in addition to the Lecture 16 notes.


The NASA polynomials for specie $i$ have the form:
$$
    \frac{C_{p,i}}{R}= a_{i1} + a_{i2} T + a_{i3} T^2 + a_{i4} T^3 + a_{i5} T^4
$$

$$
    \frac{H_{i}}{RT} = a_{i1} + a_{i2} \frac{T}{2} + a_{i3} \frac{T^2}{3} + a_{i4} \frac{T^3}{4} + a_{i5} \frac{T^4}{5} + \frac{a_{i6}}{T}
$$

$$
    \frac{S_{i}}{R}  = a_{i1} \ln(T) + a_{i2} T + a_{i3} \frac{T^2}{2} + a_{i4} \frac{T^3}{3} + a_{i5} \frac{T^4}{4} + a_{i7}
$$

where $a_{i1}$, $a_{i2}$, $a_{i3}$, $a_{i4}$, $a_{i5}$, $a_{i6}$, and $a_{i7}$ are the numerical coefficients supplied in NASA thermodynamic files. 

In [12]:
with open('thermo.txt','r') as f:
    lines=f.readlines()
    

In [18]:
molecules={}
for line in lines[5:]:
    word=line.split()
    if word[0]=="END":
        break
    print(line)
    if word[-1]=="1":
        current_molecule=word[0]
        molecules[current_molecule]={}
        molecules[current_molecule]["T_Min"]=word[-4]
        molecules[current_molecule]["T_Max"]=word[-3]
        molecules[current_molecule]["T_Between"]=word[-2]
        molecules[current_molecule]["coef_r"]=[]
        molecules[current_molecule]["coef_p"]=[]
        continue
    if word[-1]=="2":
        for i in range(5):
            molecules[current_molecule]["coef_r"].append(line[15*i:15*(i+1)])
        continue
    if word[-1]=="3":
        for i in range(2):
            molecules[current_molecule]["coef_r"].append(line[15*i:15*(i+1)])
        for i in range(2,5):
            molecules[current_molecule]["coef_p"].append(line[15*i:15*(i+1)])
        continue
    if word[-1]=="4":
        for i in range(5):
            molecules[current_molecule]["coef_p"].append(line[15*i:15*(i+1)])

  
        
        
        

O                 L 1/90O   1               G   200.000  3500.000  1000.000    1

 2.56942078E+00-8.59741137E-05 4.19484589E-08-1.00177799E-11 1.22833691E-15    2

 2.92175791E+04 4.78433864E+00 3.16826710E+00-3.27931884E-03 6.64306396E-06    3

-6.12806624E-09 2.11265971E-12 2.91222592E+04 2.05193346E+00                   4

O2                TPIS89O   2               G   200.000  3500.000  1000.000    1

 3.28253784E+00 1.48308754E-03-7.57966669E-07 2.09470555E-10-2.16717794E-14    2

-1.08845772E+03 5.45323129E+00 3.78245636E+00-2.99673416E-03 9.84730201E-06    3

-9.68129509E-09 3.24372837E-12-1.06394356E+03 3.65767573E+00                   4

H                 L 7/88H   1               G   200.000  3500.000  1000.000    1

 2.50000001E+00-2.30842973E-11 1.61561948E-14-4.73515235E-18 4.98197357E-22    2

 2.54736599E+04-4.46682914E-01 2.50000000E+00 7.05332819E-13-1.99591964E-15    3

 2.30081632E-18-9.27732332E-22 2.54736599E+04-4.46682853E-01                   4

H2              

In [39]:
import xml.etree.cElementTree as ET
p=ET.Element("ctml")
cp=ET.SubElement(p,"phase",id="gri30")
molecules_arr=ET.SubElement(cp,"speciesArray",datasrc="#species_data")
molecules_list=""
for m in molecules.keys():
    molecules_list+=m+" "
molecules_arr.text=molecules_list

data=ET.SubElement(p,"speciesData",id="species_data")

for m in molecules.keys():
    species = ET.SubElement(data, "species", name=m)
    thermo = ET.SubElement(species, "thermo")

    # each temperature range, use a 
    # sub-field with the minimum and maximum temperature as attributes
    NASA = ET.SubElement(thermo, "NASA", Tmax=molecules[m]['T_Max'],
                        Tmin=molecules[m]['T_Between'])

    # floatArray field that contains 
    #comma-separated string of each coefficient
    floatArray = ET.SubElement(NASA, "floatArray", name="coeffs", size="7")
    # high-temperature range
    coeff = molecules[m]['coef_r'][0]
    for c in molecules[m]['coef_p'][1:]:
        coeff += ',' + c
    floatArray.text = coeff

    # low-temperature range
    NASA = ET.SubElement(thermo, "NASA", Tmax=molecules[m]['T_Between'],Tmin=molecules[m]['T_Min'])
    floatArray = ET.SubElement(NASA, "floatArray", name="coeffs", size="7")
    coeff = molecules[m]['coef_r'][0]
    for c in molecules[m]['coef_p'][1:]:
        coeff += ',' + c
    floatArray.text = coeff

# write into xml file
tree = ET.ElementTree(p)
tree.write("thermo.xml")

b'<ctml><phase id="gri30"><speciesArray datasrc="#species_data">O O2 H H2 OH H2O HO2 H2O2 </speciesArray></phase><speciesData id="species_data"><species name="O"><thermo><NASA Tmax="3500.000" Tmin="1000.000"><floatArray name="coeffs" size="7"> 2.56942078E+00,-3.27931884E-03, 6.64306396E-06,-6.12806624E-09, 2.11265971E-12, 2.91222592E+04, 2.05193346E+00,               </floatArray></NASA><NASA Tmax="1000.000" Tmin="200.000"><floatArray name="coeffs" size="7"> 2.56942078E+00,-3.27931884E-03, 6.64306396E-06,-6.12806624E-09, 2.11265971E-12, 2.91222592E+04, 2.05193346E+00,               </floatArray></NASA></thermo></species><species name="O2"><thermo><NASA Tmax="3500.000" Tmin="1000.000"><floatArray name="coeffs" size="7"> 3.28253784E+00,-2.99673416E-03, 9.84730201E-06,-9.68129509E-09, 3.24372837E-12,-1.06394356E+03, 3.65767573E+00,               </floatArray></NASA><NASA Tmax="1000.000" Tmin="200.000"><floatArray name="coeffs" size="7"> 3.28253784E+00,-2.99673416E-03, 9.84730201E-06,-9.68

### Some Notes on `thermo.txt`
The first 7 numbers starting on the second line of each species entry (five of the second line and the first two of the third line) are the seven coefficients ($a_{i1}$ through $a_{i7}$, respectively) for the high-temperature range (above 1000 K, the upper boundary is specified on the first line of the species entry). 

The next seven numbers are the coefficients ($a_{i1}$ through $a_{i7}$, respectively) for the low-temperature range (below 1000 K, the lower boundary is specified on the first line of the species entry).

### Additional Specifications
Your final .xml file should contain the following specifications:

1. A `speciesArray` field that contains a space-separated list of all of the species present in the file.
2. Each species contains a `species` field with a `name` attribute as the species name.

    1. For each temperature range, use a sub-field with the minimum and maximum temperature as attributes.
    2. `floatArray` field that contains comma-separated string of each coefficient.
    
You can reference the `example_thermo.xml` file for an example .xml output.

**Hint**: First parse the file into a Python dictionary. 